# Analyzing CIA Factbook Data Using SQL

In this project, we are going to explore the CIA Factbook data which stores in the SQLite database. Factbook data contains information regarding worldwide demographics including population, population growth, area, etc.

The description of all columns is below.

| Columns    | Description                                                  |
| ----------------- | ------------------------------------------------------------ |
| name              | The name of the country.                                     |
| area              | The country's total area (both land and water).              |
| area_land         | The country's land area in square kilometers                 |
| area_water        | The country's waterarea in square kilometers.                |
| population        | The country's population.                                    |
| population_growth | The country's population growth as a percentage.             |
| birth_rate        | The country's birth rate, or the number of births a year per 1,000 people. |
| death_rate        | The country's death rate, or the number of death a year per 1,000 people. |


## Loading SQL and Factbook data

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Exploring Tables in Database

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table'

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Overview of the Data

In [3]:
%%sql
SELECT *
FROM facts
limit 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics

In [4]:
%%sql
SELECT MIN(population) AS minimum_population, MAX(population) AS maximum_population, MIN(population_growth) AS minimum_population_growth, MAX(population_growth) AS maximum_population_growth
FROM facts;

 * sqlite:///factbook.db
Done.


minimum_population,maximum_population,minimum_population_growth,maximum_population_growth
0,7256490011,0.0,4.02


There're a few things stick out from the summary statistics.

- There's a country with 0 population which is the lowest number.
- There's a country with a 7,256,490,011 (7.2 billion) population which is the highest number.
- There's a country with 0 population growth which is the lowest number.
- There's a country with 4.02 population growth which is the highest number.

We're going to explore further to find out which of these countries are.

## Exploring Outliers

In [5]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


The row with 0 population is Antarctica

In [6]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The row with the highest population is the row containing the whole world population.

In [7]:
%%sql
SELECT *
FROM facts
WHERE population_growth = (SELECT MIN(population_growth) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
190,vt,Holy See (Vatican City),0,0,0,842,0.0,None,None,None
200,ck,Cocos (Keeling) Islands,14,14,0,596,0.0,None,None,None
207,gl,Greenland,2166086,2166086,None,57733,0.0,14.48,8.49,5.98
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None


The rows with 0 population growth are Holy See (Vatican City), Cocos (Keeling) Islands, Greenland, and the Pitcairn Islands.

In [8]:
%%sql
SELECT *
FROM facts
WHERE population_growth = (SELECT MAX(population_growth) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


The row with 4.02 is in South Sudan which is the highest population growth.

## Revised Sumary Statistics

In [12]:
%%sql
SELECT MIN(population) AS minimum_population, MAX(population) AS maximum_population, MIN(population_growth) AS minimum_population_growth, MAX(population_growth) AS maximum_population_growth
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


minimum_population,maximum_population,minimum_population_growth,maximum_population_growth
0,1367485388,0.0,4.02


After excluding a row containing the world data, the maximum population decrease from around 7.2 billion to 1.3 billion.

## Exploing Average Population and Area

In [14]:
%%sql
SELECT AVG(population) AS average_population, AVG(area) AS average_area
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


average_population,average_area
32242666.56846473,555093.546184739


The average population is around 32 million and the average area is around 555 thousand square kilometers.

## Finding Densely Populated Countries

To find countries with high population density, we need to add conditions to the query which are below.
- The countries that have population more than average
- The countries that have areas less than average

In [16]:
%%sql
SELECT *
FROM facts
WHERE population >= (SELECT AVG(population) FROM facts)
    AND area <= (SELECT AVG(area) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


There're 7 countries that have high population density.